# Homework 2

## Message from your Prof
> **Remember for the best way to learn this content, and maximize your learning experience, you must implement these models yourself and only use imports for checking your work**

You can only import classification_report from sklearn evaluation metrics (you do not need to implement classification report), but you have to implement the train_test_split, and the classifiers yourselves. You will need to use numpy or pandas as inputs for your models. You should only use the imports listed below to check your work.

Students that do not practice their own implementations **will be cooked** in their skill assessments. I do not want to hear students complaining they did poorly because the homeworks did not reflect the skill assessments. YOU HAVE BEEN WARNED!!!

<br>
In this assignment, we will be building a Naïve Bayes classifier and a SVM model for the productivity satisfaction of [the given dataset](https://archive.ics.uci.edu/ml/datasets/Productivity+Prediction+of+Garment+Employees), the productivity of garment employees.

## Background
The Garment Industry is one of the key examples of the industrial globalization of this modern era. It is a highly labour-intensive industry with lots of manual processes. Satisfying the huge global demand for garment products is mostly dependent on the production and delivery performance of the employees in the garment manufacturing companies. So, it is highly desirable among the decision makers in the garments industry to track, analyse and predict the productivity performance of the working teams in their factories.

## Dataset Attribute Information

1. **date**: Date in MM-DD-YYYY
2. **day**: Day of the Week
3. **quarter** : A portion of the month. A month was divided into four quarters
4. **department** : Associated department with the instance
5. **team_no** : Associated team number with the instance
6. **no_of_workers** : Number of workers in each team
7. **no_of_style_change** : Number of changes in the style of a particular product
8. **targeted_productivity** : Targeted productivity set by the Authority for each team for each day.
9. **smv** : Standard Minute Value, it is the allocated time for a task
10. **wip** : Work in progress. Includes the number of unfinished items for products
11. **over_time** : Represents the amount of overtime by each team in minutes
12. **incentive** : Represents the amount of financial incentive (in BDT) that enables or motivates a particular course of action.
13. **idle_time** : The amount of time when the production was interrupted due to several reasons
14. **idle_men** : The number of workers who were idle due to production interruption
15. **actual_productivity** : The actual % of productivity that was delivered by the workers. It ranges from 0-1.

### Libraries that can be used: numpy, scipy, pandas, scikit-learn, cvxpy, imbalanced-learn
Any libraries used in the discussion materials are also allowed.

#### Other Notes

 - Don't worry about not being able to achieve high accuracy, it is neither the goal nor the grading standard of this assignment. <br >
 - If not specified, you are not required to do hyperparameter tuning, but feel free to do so if you'd like.

#### Trouble Shooting
In case you have trouble installing and using imbalanced-learn(imblearn) <br >
Run the below code cell, then go to the selection bar at top: Kernel > Restart. <br >
Then try `import imblearn` to see if things work.

In [287]:
# import platform
# display(platform.system())
# import os
# file_download_link = 'https://www.dropbox.com/scl/fi/j1dxtqjerbdbl81e05nvp/hw4data.zip?rlkey=8mkxz4j8lngziok6on782a8u4&dl=0'
# if os.name == 'nt':
#     print('Please download your dataset here:', file_download_link)
# else:
#     # We need to first download the data here:
#     !wget -O data.zip "$file_download_link" -o /dev/null
#     !unzip data.zip > /dev/null
# y

In [288]:
# !sed 's/,/\t/g' garments_worker_productivity.csv > garments_worker_productivity.tsv

In [289]:
# If your data is on google drive then uncomment the code below to access
# your google drive.
#from google.colab import drive
#drive.mount('/content/drive')

In [290]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install imbalanced-learn delayed


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip


# Exercises

## Exercise 1 - General Data Preprocessing (20 points)

Our dataset needs cleaning before building any models. Some of the cleaning tasks are common in general, but depends on what kind of models we are building, sometimes we have to do additional processing. These additional tasks will be mentioned in each of the remaining two exercises later.

Note that **we will be using this processed data from exercise 1 in each of the remaining two exercises**.

For convenience, here are the attributes that we would treat them as **categorical attributes**: `day`, `quarter`, `department`, and `team`.

Realize that `quarter` is not referring to certain months of the year, but certain days of the month. "Quarter1" represents days 1-7, "Quarter2" days 8-14, "Quarter3" days 15-21, "Quarter4" days 22-28, and "Quarter5" days 29-31

 - Drop the column `date`.
 - For each of the categorical attributes, **print out** all the unique elements.
 - For each of the categorical attributes, remap the duplicated items, if you find there are typos or spaces among the duplicated items.
     - For example, "a" and "a " should be the same, so we need to update "a " to be "a".
     - Another example, "apple" and "appel" should be the same, so you should update "appel" to be "apple".
     

 - Create another column named `satisfied` that records the productivity performance. The behavior defined as follows. **This is the dependent variable we'd like to classify in this assignment.**
     - Return True or 1 if `actual_productivity` is equal to or greater than `targeted_productivity`. Otherwise, return False or 0, which means the team fails to meet the expected performance.
 - Drop the columns `actual_productivity` and `targeted_productivity`.


 - Find and **print out** which columns/attributes that have empty vaules, e.g., NA, NaN, null, None.
 - You must use `df.describe()` or `df.info()` to display the data after preprocessing. **No credit** will be given if this step is omitted.
 - Fill the empty values with 0.


In [291]:
import pandas as pd
# If put the data(.csv) under the same folder, you could use
df = pd.read_csv('garments_worker_productivity.csv')

In [292]:
df

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,1/1/2015,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,1/1/2015,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,1/1/2015,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,3/11/2015,Quarter2,finishing,Wednesday,10,0.75,2.90,NaN,960,0,0.0,0,0,8.0,0.628333
1193,3/11/2015,Quarter2,finishing,Wednesday,8,0.70,3.90,NaN,960,0,0.0,0,0,8.0,0.625625
1194,3/11/2015,Quarter2,finishing,Wednesday,7,0.65,3.90,NaN,960,0,0.0,0,0,8.0,0.625625
1195,3/11/2015,Quarter2,finishing,Wednesday,9,0.75,2.90,NaN,1800,0,0.0,0,0,15.0,0.505889


In [293]:
columns=list(df.columns[1:])
df=df[columns]
df

,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,Quarter2,finishing,Wednesday,10,0.75,2.90,NaN,960,0,0.0,0,0,8.0,0.628333
1193,Quarter2,finishing,Wednesday,8,0.70,3.90,NaN,960,0,0.0,0,0,8.0,0.625625
1194,Quarter2,finishing,Wednesday,7,0.65,3.90,NaN,960,0,0.0,0,0,8.0,0.625625
1195,Quarter2,finishing,Wednesday,9,0.75,2.90,NaN,1800,0,0.0,0,0,15.0,0.505889


In [294]:
for i in ['day','quarter','department','team']:
    print(df[i].unique())

['Thursday' 'Saturday' 'Sunday' 'Monday' 'Tuesday' 'Wednesday']
['Quarter1' 'Quarter2' 'Quarter3' 'Quarter4' 'Quarter5']
['sweing' 'finishing ' 'finishing']
[ 8  1 11 12  6  7  2  3  9 10  5  4]


In [295]:
df['day']=df['day'].transform(lambda x: x.strip())
df['day']

/var/folders/n_/1hzw47553v1bvg9z8kct6rd40000gn/T/ipykernel_44190/1951944645.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day']=df['day'].transform(lambda x: x.strip())


0        Thursday
1        Thursday
2        Thursday
3        Thursday
4        Thursday
          ...    
1192    Wednesday
1193    Wednesday
1194    Wednesday
1195    Wednesday
1196    Wednesday
Name: day, Length: 1197, dtype: object

In [296]:
df['department']=df['department'].transform(lambda x: 'sewing' if x=='sweing' else x.strip())
df['department']

/var/folders/n_/1hzw47553v1bvg9z8kct6rd40000gn/T/ipykernel_44190/3522869424.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['department']=df['department'].transform(lambda x: 'sewing' if x=='sweing' else x.strip())


0          sewing
1       finishing
2          sewing
3          sewing
4          sewing
          ...    
1192    finishing
1193    finishing
1194    finishing
1195    finishing
1196    finishing
Name: department, Length: 1197, dtype: object

In [297]:
df['satisfied']= df['actual_productivity']>=df['targeted_productivity']
df['satisfied']

0        True
1        True
2        True
3        True
4        True
        ...  
1192    False
1193    False
1194    False
1195    False
1196    False
Name: satisfied, Length: 1197, dtype: bool

In [298]:
df.drop(["actual_productivity", "targeted_productivity"], axis=1)

,quarter,department,day,team,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,satisfied
0,Quarter1,sewing,Thursday,8,26.16,1108.0,7080,98,0.0,0,0,59.0,True
1,Quarter1,finishing,Thursday,1,3.94,NaN,960,0,0.0,0,0,8.0,True
2,Quarter1,sewing,Thursday,11,11.41,968.0,3660,50,0.0,0,0,30.5,True
3,Quarter1,sewing,Thursday,12,11.41,968.0,3660,50,0.0,0,0,30.5,True
4,Quarter1,sewing,Thursday,6,25.90,1170.0,1920,50,0.0,0,0,56.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,Quarter2,finishing,Wednesday,10,2.90,NaN,960,0,0.0,0,0,8.0,False
1193,Quarter2,finishing,Wednesday,8,3.90,NaN,960,0,0.0,0,0,8.0,False
1194,Quarter2,finishing,Wednesday,7,3.90,NaN,960,0,0.0,0,0,8.0,False
1195,Quarter2,finishing,Wednesday,9,2.90,NaN,1800,0,0.0,0,0,15.0,False


In [299]:
df['wip'].isna()

0       False
1        True
2       False
3       False
4       False
        ...  
1192     True
1193     True
1194     True
1195     True
1196     True
Name: wip, Length: 1197, dtype: bool

In [300]:
print('wip')

wip


In [301]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   quarter                1197 non-null   object 
 1   department             1197 non-null   object 
 2   day                    1197 non-null   object 
 3   team                   1197 non-null   int64  
 4   targeted_productivity  1197 non-null   float64
 5   smv                    1197 non-null   float64
 6   wip                    691 non-null    float64
 7   over_time              1197 non-null   int64  
 8   incentive              1197 non-null   int64  
 9   idle_time              1197 non-null   float64
 10  idle_men               1197 non-null   int64  
 11  no_of_style_change     1197 non-null   int64  
 12  no_of_workers          1197 non-null   float64
 13  actual_productivity    1197 non-null   float64
 14  satisfied              1197 non-null   bool   
dtypes: b

In [302]:
df['wip']=df['wip'].fillna(0)
df['wip']

0       1108.0
1          0.0
2        968.0
3        968.0
4       1170.0
         ...  
1192       0.0
1193       0.0
1194       0.0
1195       0.0
1196       0.0
Name: wip, Length: 1197, dtype: float64

## Exercise 2 - Naïve Bayes Classifier (40 points in total)

### Exercise 2.1 - Additional Data Preprocessing (10 points)

To build a Naïve Bayes Classifier, we need to further encode our categorical variables.

 - For each of the **categorical attributes**, encode the set of categories to be **0 ~ (n_classes - 1)**.
     - For example, \["paris", "paris", "tokyo", "amsterdam"\] should be encoded as \[1, 1, 2, 0\].
     - Note that the order does not really matter, i.e., \[0, 0, 1, 2\] also works. But you have to start with 0 in your encodings.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20.
 - You **must** show the first five row of your encoded dataset, as well as the shape of your train test split. **No credit** will be given if this step is omitted.

In [303]:
df['day'].unique()

array(['Thursday', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday'],
      dtype=object)

In [304]:
# Remember to continue the task with your processed data from Exercise 1
df['day']=df['day'].transform(lambda x: 0 if x == 'Thursday' else 1 if x == 'Saturday' else 2 if x == 'Sunday' else 3 if x == 'Monday' else 4 if x == 'Tuesday' else 5)
df['day']

0       0
1       0
2       0
3       0
4       0
       ..
1192    5
1193    5
1194    5
1195    5
1196    5
Name: day, Length: 1197, dtype: int64

In [305]:
df['quarter'].unique()

array(['Quarter1', 'Quarter2', 'Quarter3', 'Quarter4', 'Quarter5'],
      dtype=object)

In [306]:
df['quarter']=df['quarter'].transform(lambda x: 0 if x=='Quarter1' else 1 if x=='Quarter2' else 2 if x=='Quarter3' else 3 if x=='Quarter4' else 4)
df['quarter']

0       0
1       0
2       0
3       0
4       0
       ..
1192    1
1193    1
1194    1
1195    1
1196    1
Name: quarter, Length: 1197, dtype: int64

In [307]:
df['department'].unique()

array(['sewing', 'finishing'], dtype=object)

In [308]:
df['department']=df['department'].transform(lambda x: 0 if x=='sewing' else 1)
df['department'] 

0       0
1       1
2       0
3       0
4       0
       ..
1192    1
1193    1
1194    1
1195    1
1196    1
Name: department, Length: 1197, dtype: int64

In [309]:
poss=list(df['team'].unique())
df['team']=df['team'].transform(lambda x: poss.index(x))
df['team']

0       0
1       1
2       2
3       3
4       4
       ..
1192    9
1193    0
1194    5
1195    8
1196    4
Name: team, Length: 1197, dtype: int64

In [310]:


train_df = df.iloc[:int(len(df) * 0.8)]
test_df  = df.iloc[int(len(df) * 0.8):] 

print(train_df.shape)
print(test_df.shape)


(957, 15)
(240, 15)


In [311]:
train_df.head()

,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity,satisfied
0,0,0,0,0,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725,True
1,0,1,0,1,0.75,3.94,0.0,960,0,0.0,0,0,8.0,0.886500,True
2,0,0,0,2,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570,True
3,0,0,0,3,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570,True
4,0,0,0,4,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382,True


In [312]:
test_df.head()

,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity,satisfied
957,3,0,0,11,0.80,30.10,437.0,7080,32,0.0,0,2,59.0,0.495618,False
958,3,0,0,10,0.35,27.48,413.0,6840,38,0.0,0,1,57.0,0.449965,True
959,3,1,0,9,0.70,2.90,0.0,3360,0,0.0,0,0,8.0,0.410833,False
960,3,1,0,8,0.75,2.90,0.0,960,0,0.0,0,0,8.0,0.407813,False
961,3,0,0,1,0.35,26.66,1164.0,6600,23,0.0,0,2,55.0,0.378895,True


### Exercise 2.2 - Naïve Bayes Classifier for Categorical Attributes (15 points)

Use the categorical attributes **only**, please build a Categorical Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the **testing result** using `classification_report`.

In [313]:
# Remember to do this task with your processed data from Exercise 2.1
X=train_df[['quarter','department','day','team','satisfied']]
options= {(i, j, k, l):0 for i in range(5) for j in range(2) for k in range(6) for l in range(12)}
options2=options.copy()
P_s1=0

In [314]:
P_s1=X[X['satisfied']==True].shape[0]/(X['satisfied'].shape[0])
P_s1

0.7314524555903866

In [315]:
X

,quarter,department,day,team,satisfied
0,0,0,0,0,True
1,0,1,0,1,True
2,0,0,0,2,True
3,0,0,0,3,True
4,0,0,0,4,True
...,...,...,...,...,...
952,3,0,0,0,True
953,3,0,0,9,True
954,3,0,0,6,False
955,3,1,0,3,False


In [316]:
options

{(0, 0, 0, 0): 0,
 (0, 0, 0, 1): 0,
 (0, 0, 0, 2): 0,
 (0, 0, 0, 3): 0,
 (0, 0, 0, 4): 0,
 (0, 0, 0, 5): 0,
 (0, 0, 0, 6): 0,
 (0, 0, 0, 7): 0,
 (0, 0, 0, 8): 0,
 (0, 0, 0, 9): 0,
 (0, 0, 0, 10): 0,
 (0, 0, 0, 11): 0,
 (0, 0, 1, 0): 0,
 (0, 0, 1, 1): 0,
 (0, 0, 1, 2): 0,
 (0, 0, 1, 3): 0,
 (0, 0, 1, 4): 0,
 (0, 0, 1, 5): 0,
 (0, 0, 1, 6): 0,
 (0, 0, 1, 7): 0,
 (0, 0, 1, 8): 0,
 (0, 0, 1, 9): 0,
 (0, 0, 1, 10): 0,
 (0, 0, 1, 11): 0,
 (0, 0, 2, 0): 0,
 (0, 0, 2, 1): 0,
 (0, 0, 2, 2): 0,
 (0, 0, 2, 3): 0,
 (0, 0, 2, 4): 0,
 (0, 0, 2, 5): 0,
 (0, 0, 2, 6): 0,
 (0, 0, 2, 7): 0,
 (0, 0, 2, 8): 0,
 (0, 0, 2, 9): 0,
 (0, 0, 2, 10): 0,
 (0, 0, 2, 11): 0,
 (0, 0, 3, 0): 0,
 (0, 0, 3, 1): 0,
 (0, 0, 3, 2): 0,
 (0, 0, 3, 3): 0,
 (0, 0, 3, 4): 0,
 (0, 0, 3, 5): 0,
 (0, 0, 3, 6): 0,
 (0, 0, 3, 7): 0,
 (0, 0, 3, 8): 0,
 (0, 0, 3, 9): 0,
 (0, 0, 3, 10): 0,
 (0, 0, 3, 11): 0,
 (0, 0, 4, 0): 0,
 (0, 0, 4, 1): 0,
 (0, 0, 4, 2): 0,
 (0, 0, 4, 3): 0,
 (0, 0, 4, 4): 0,
 (0, 0, 4, 5): 0,
 (0, 0, 4, 6): 0,
 (

In [317]:
for i in options.keys():
    P_x1_s1=X[(X['quarter']==i[0])&(X['satisfied']==False)].shape[0]/X[(X['satisfied']==False)].shape[0] if X[(X['satisfied']==False)].shape[0]>0 else 0
    P_x2_s1=X[(X['department']==i[1])&(X['satisfied']==False)].shape[0]/X[(X['satisfied']==False)].shape[0] if X[(X['satisfied']==False)].shape[0]>0 else 0
    P_x3_s1=X[(X['day']==i[2])&(X['satisfied']==False)].shape[0]/X[(X['satisfied']==False)].shape[0] if X[(X['satisfied']==False)].shape[0]>0 else 0
    P_x4_s1=X[(X['team']==i[3])&(X['satisfied']==False)].shape[0]/X[(X['satisfied']==False)].shape[0] if X[(X['satisfied']==False)].shape[0]>0 else 0
    joint = X[(X['quarter']==i[0])&(X['department']==i[1])&(X['day']==i[2])&(X['team']==i[3])].shape[0]/X.shape[0] if X.shape[0]>0 else 0
    options[i]=(P_x1_s1*P_x2_s1*P_x3_s1*P_x4_s1*(1-P_s1))/joint if joint >0 else 0 
options

{(0, 0, 0, 0): 0.3841505949509363,
 (0, 0, 0, 1): 0.10564141361150749,
 (0, 0, 0, 2): 0.20167906234924157,
 (0, 0, 0, 3): 0.1248489433590543,
 (0, 0, 0, 4): 0.23049035697056178,
 (0, 0, 0, 5): 0.3169242408345224,
 (0, 0, 0, 6): 0.19207529747546814,
 (0, 0, 0, 7): 0.08643388386396067,
 (0, 0, 0, 8): 0.2689054164656554,
 (0, 0, 0, 9): 0.19207529747546814,
 (0, 0, 0, 10): 0.21128282722301497,
 (0, 0, 0, 11): 0.15366023798037454,
 (0, 0, 1, 0): 0.21566349190228004,
 (0, 0, 1, 1): 0.05930746027312701,
 (0, 0, 1, 2): 0.11322333324869702,
 (0, 0, 1, 3): 0.070090634868241,
 (0, 0, 1, 4): 0.12939809514136805,
 (0, 0, 1, 5): 0.17792238081938103,
 (0, 0, 1, 6): 0.10783174595114002,
 (0, 0, 1, 7): 0.048524285678013,
 (0, 0, 1, 8): 0.15096444433159603,
 (0, 0, 1, 9): 0.10783174595114002,
 (0, 0, 1, 10): 0.11861492054625403,
 (0, 0, 1, 11): 0.08626539676091202,
 (0, 0, 2, 0): 0.35045317434120504,
 (0, 0, 2, 1): 0.09637462294383138,
 (0, 0, 2, 2): 0.18398791652913268,
 (0, 0, 2, 3): 0.113897281660891

In [318]:
options2

{(0, 0, 0, 0): 0,
 (0, 0, 0, 1): 0,
 (0, 0, 0, 2): 0,
 (0, 0, 0, 3): 0,
 (0, 0, 0, 4): 0,
 (0, 0, 0, 5): 0,
 (0, 0, 0, 6): 0,
 (0, 0, 0, 7): 0,
 (0, 0, 0, 8): 0,
 (0, 0, 0, 9): 0,
 (0, 0, 0, 10): 0,
 (0, 0, 0, 11): 0,
 (0, 0, 1, 0): 0,
 (0, 0, 1, 1): 0,
 (0, 0, 1, 2): 0,
 (0, 0, 1, 3): 0,
 (0, 0, 1, 4): 0,
 (0, 0, 1, 5): 0,
 (0, 0, 1, 6): 0,
 (0, 0, 1, 7): 0,
 (0, 0, 1, 8): 0,
 (0, 0, 1, 9): 0,
 (0, 0, 1, 10): 0,
 (0, 0, 1, 11): 0,
 (0, 0, 2, 0): 0,
 (0, 0, 2, 1): 0,
 (0, 0, 2, 2): 0,
 (0, 0, 2, 3): 0,
 (0, 0, 2, 4): 0,
 (0, 0, 2, 5): 0,
 (0, 0, 2, 6): 0,
 (0, 0, 2, 7): 0,
 (0, 0, 2, 8): 0,
 (0, 0, 2, 9): 0,
 (0, 0, 2, 10): 0,
 (0, 0, 2, 11): 0,
 (0, 0, 3, 0): 0,
 (0, 0, 3, 1): 0,
 (0, 0, 3, 2): 0,
 (0, 0, 3, 3): 0,
 (0, 0, 3, 4): 0,
 (0, 0, 3, 5): 0,
 (0, 0, 3, 6): 0,
 (0, 0, 3, 7): 0,
 (0, 0, 3, 8): 0,
 (0, 0, 3, 9): 0,
 (0, 0, 3, 10): 0,
 (0, 0, 3, 11): 0,
 (0, 0, 4, 0): 0,
 (0, 0, 4, 1): 0,
 (0, 0, 4, 2): 0,
 (0, 0, 4, 3): 0,
 (0, 0, 4, 4): 0,
 (0, 0, 4, 5): 0,
 (0, 0, 4, 6): 0,
 (

In [319]:
for i in options2.keys():
    P_x1_s1=X[(X['quarter']==i[0])&(X['satisfied']==True)].shape[0]/X[(X['satisfied']==True)].shape[0] if X[(X['satisfied']==True)].shape[0]>0 else 0
    P_x2_s1=X[(X['department']==i[1])&(X['satisfied']==True)].shape[0]/X[(X['satisfied']==True)].shape[0] if X[(X['satisfied']==True)].shape[0]>0 else 0
    P_x3_s1=X[(X['day']==i[2])&(X['satisfied']==True)].shape[0]/X[(X['satisfied']==True)].shape[0] if X[(X['satisfied']==True)].shape[0]>0 else 0
    P_x4_s1=X[(X['team']==i[3])&(X['satisfied']==True)].shape[0]/X[(X['satisfied']==True)].shape[0] if X[(X['satisfied']==True)].shape[0]>0 else 0
    joint = X[(X['quarter']==i[0])&(X['department']==i[1])&(X['day']==i[2])&(X['team']==i[3])].shape[0]/X.shape[0] if X.shape[0]>0 else 0 
    options2[i]=(P_x1_s1*P_x2_s1*P_x3_s1*P_x4_s1*P_s1)/joint if joint>0 else 0 
options2

{(0, 0, 0, 0): 0.6825101807580173,
 (0, 0, 0, 1): 1.0095463090379004,
 (0, 0, 0, 2): 0.7109481049562679,
 (0, 0, 0, 3): 0.9242325364431484,
 (0, 0, 0, 4): 0.7536049912536442,
 (0, 0, 0, 5): 0.6682912186588919,
 (0, 0, 0, 6): 0.9668894227405244,
 (0, 0, 0, 7): 0.9242325364431484,
 (0, 0, 0, 8): 0.7820429154518947,
 (0, 0, 0, 9): 0.8531377259475216,
 (0, 0, 0, 10): 0.7251670670553934,
 (0, 0, 0, 11): 0.9526704606413992,
 (0, 0, 1, 0): 0.6766264723032068,
 (0, 0, 1, 1): 1.00084332361516,
 (0, 0, 1, 2): 0.704819241982507,
 (0, 0, 1, 3): 0.9162650145772592,
 (0, 0, 1, 4): 0.7471083965014574,
 (0, 0, 1, 5): 0.6625300874635567,
 (0, 0, 1, 6): 0.9585541690962095,
 (0, 0, 1, 7): 0.9162650145772592,
 (0, 0, 1, 8): 0.7753011661807577,
 (0, 0, 1, 9): 0.8457830903790086,
 (0, 0, 1, 10): 0.7189156268221573,
 (0, 0, 1, 11): 0.9444577842565594,
 (0, 0, 2, 0): 0.6413242215743439,
 (0, 0, 2, 1): 0.9486254110787169,
 (0, 0, 2, 2): 0.6680460641399415,
 (0, 0, 2, 3): 0.868459883381924,
 (0, 0, 2, 4): 0.708

In [320]:
X=test_df[['quarter','department','day','team']]
row=X.iloc[0]
row.iloc[1]
y_test=test_df['satisfied']
type(y_test)

pandas.core.series.Series

In [321]:
ypred=[]
for i in range(len(X)):
    row=X.iloc[i]
    tup=(int(row.iloc[0]),int(row.iloc[1]),int(row.iloc[2]),int(row.iloc[3]))
    if options2[tup]>=0.5:
        ypred.append(1)
    else:
        ypred.append(0)

In [322]:
ypred = pd.Series(ypred)
type(ypred)

pandas.core.series.Series

In [323]:
from sklearn.metrics import classification_report

In [324]:
print(classification_report(y_test, ypred))

              precision    recall  f1-score   support

       False       0.53      0.26      0.35        65
        True       0.77      0.91      0.84       175

    accuracy                           0.74       240
   macro avg       0.65      0.59      0.59       240
weighted avg       0.70      0.74      0.70       240



### Exercise 2.3 - Naïve Bayes Classifier for Numerical Attributes (15 points)

Use the numerical attributes **only**, please build a Gaussian Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the **testing result** using `classification_report`.

**Remember to scale your data. The scaling method is up to you.**

In [325]:
list(train_df.columns)

['quarter',
 'department',
 'day',
 'team',
 'targeted_productivity',
 'smv',
 'wip',
 'over_time',
 'incentive',
 'idle_time',
 'idle_men',
 'no_of_style_change',
 'no_of_workers',
 'actual_productivity',
 'satisfied']

In [326]:
# Remember to do this task with your processed data from Exercise 2.1
X= train_df[list(train_df.columns)[4:]]
X=X.drop('actual_productivity',axis=1)
X= X.drop('targeted_productivity',axis=1)
X

,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,satisfied
0,26.16,1108.0,7080,98,0.0,0,0,59.0,True
1,3.94,0.0,960,0,0.0,0,0,8.0,True
2,11.41,968.0,3660,50,0.0,0,0,30.5,True
3,11.41,968.0,3660,50,0.0,0,0,30.5,True
4,25.90,1170.0,1920,50,0.0,0,0,56.0,True
...,...,...,...,...,...,...,...,...,...
952,29.40,916.0,6960,56,0.0,0,2,58.0,True
953,21.82,1591.0,3240,0,0.0,0,1,52.0,True
954,30.33,398.0,6960,0,0.0,0,1,58.0,False
955,4.60,0.0,3780,0,0.0,0,0,9.0,False


In [327]:

# b=X['smv'].unique()
# c=X['wip'].unique()
# d=X['over_time'].unique()
# e=X['incentive'].unique()
# f=X['idle_time'].unique()
# g=X['idle_men'].unique()
# h= X['no_of_style_change'].unique()
# i = X['no_of_workers'].unique()
# options = {(x1, x2, x3, x4, x5, x6, x7, x8, x9): 0
#            for x1 in a
#            for x2 in b
#            for x3 in c
#            for x4 in d
#            for x5 in e
#            for x6 in f
#            for x7 in g
#            for x8 in h
#            for x9 in i}
# options


In [328]:
P_s1

0.7314524555903866

In [329]:
import numpy as np

In [ ]:
temp=X[X['satisfied']==True]
temp2=X[X['satisfied']==False]
P_x1_s1 = (np.exp((-(temp['smv']-temp['smv'].mean())**2) /
                  (2*np.var(temp['smv'])))) / (np.sqrt(2*np.pi*np.var(temp['smv'])))

P_x2_s1 = (np.exp((-(temp['wip']-temp['wip'].mean())**2) /
                  (2*np.var(temp['wip'])))) / (np.sqrt(2*np.pi*np.var(temp['wip'])))

P_x3_s1 = (np.exp((-(temp['over_time']-temp['over_time'].mean())**2) /
                  (2*np.var(temp['over_time'])))) / (np.sqrt(2*np.pi*np.var(temp['over_time'])))

P_x4_s1 = (np.exp((-(temp['incentive']-temp['incentive'].mean())**2) /
                  (2*np.var(temp['incentive'])))) / (np.sqrt(2*np.pi*np.var(temp['incentive'])))

P_x5_s1 = (np.exp((-(temp['idle_time']-temp['idle_time'].mean())**2) /
                  (2*np.var(temp['idle_time'])))) / (np.sqrt(2*np.pi*np.var(temp['idle_time'])))

P_x6_s1 = (np.exp((-(temp['idle_men']-temp['idle_men'].mean())**2) /
                  (2*np.var(temp['idle_men'])))) / (np.sqrt(2*np.pi*np.var(temp['idle_men'])))

P_x7_s1 = (np.exp((-(temp['no_of_style_change']-temp['no_of_style_change'].mean())**2) /
                  (2*np.var(temp['no_of_style_change'])))) / (np.sqrt(2*np.pi*np.var(temp['no_of_style_change'])))

P_x8_s1 = (np.exp((-(temp['no_of_workers']-temp['no_of_workers'].mean())**2) /
                  (2*np.var(temp['no_of_workers'])))) / (np.sqrt(2*np.pi*np.var(temp['no_of_workers'])))
P_x1_s2 = (np.exp((-(temp2['smv']-temp2['smv'].mean())**2) /
                  (2*np.var(temp2['smv'])))) / (np.sqrt(2*np.pi*np.var(temp2['smv'])))

P_x2_s2 = (np.exp((-(temp2['wip']-temp2['wip'].mean())**2) /
                  (2*np.var(temp2['wip'])))) / (np.sqrt(2*np.pi*np.var(temp2['wip'])))

P_x3_s2 = (np.exp((-(temp2['over_time']-temp2['over_time'].mean())**2) /
                  (2*np.var(temp2['over_time'])))) / (np.sqrt(2*np.pi*np.var(temp2['over_time'])))

P_x4_s2 = (np.exp((-(temp2['incentive']-temp2['incentive'].mean())**2) /
                  (2*np.var(temp2['incentive'])))) / (np.sqrt(2*np.pi*np.var(temp2['incentive'])))

P_x5_s2 = (np.exp((-(temp2['idle_time']-temp2['idle_time'].mean())**2) /
                  (2*np.var(temp2['idle_time'])))) / (np.sqrt(2*np.pi*np.var(temp2['idle_time'])))

P_x6_s2 = (np.exp((-(temp2['idle_men']-temp2['idle_men'].mean())**2) /
                  (2*np.var(temp2['idle_men'])))) / (np.sqrt(2*np.pi*np.var(temp2['idle_men'])))

P_x7_s2 = (np.exp((-(temp2['no_of_style_change']-temp2['no_of_style_change'].mean())**2) /
                  (2*np.var(temp2['no_of_style_change'])))) / (np.sqrt(2*np.pi*np.var(temp2['no_of_style_change'])))

P_x8_s2 = (np.exp((-(temp2['no_of_workers']-temp2['no_of_workers'].mean())**2) /
                  (2*np.var(temp2['no_of_workers'])))) / (np.sqrt(2*np.pi*np.var(temp2['no_of_workers'])))
post1=[P_x1_s1,P_x2_s1,P_x3_s1,P_x4_s1,P_x5_s1,P_x6_s1,P_x7_s1,P_x8_s1]
post2=[P_x1_s2,P_x2_s2,P_x3_s2,P_x4_s2,P_x5_s2,P_x6_s2,P_x7_s2,P_x8_s2]
like_s1 = np.prod(post1)  
like_s2 = np.prod(post2) 
eps = 1e-9
P_data = (P_s1 * like_s1 )+ ((1 - P_s1) * like_s2)
P_data

np.float64(0.0)

## Exercies 3 - SVM Classifier (35 points in total)

### Exercise 3.1 - Additional Data Preprocessing (10 points)

To build a SVM Classifier, we need a different encoding for our categorical variables.

 - For each of the **categorical attributes**, encode them with **one-hot encoding**.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20.
 - You **must** show the first five row of your encoded dataset, as well as the shape of your train test split. **No credit** will be given if this step is omitted.

In [262]:
df=df.drop(columns=['actual_productivity','targeted_productivity'])

In [263]:
df=df[['quarter','department','day','team','satisfied']]
df


,quarter,department,day,team,satisfied
0,0,0,0,0,True
1,0,1,0,1,True
2,0,0,0,2,True
3,0,0,0,3,True
4,0,0,0,4,True
...,...,...,...,...,...
1192,1,1,5,9,False
1193,1,1,5,0,False
1194,1,1,5,5,False
1195,1,1,5,8,False


In [264]:
# Remember to continue the task with your processed data from Exercise 1
quarter = pd.DataFrame(columns=['q1','q2','q3','q4','q5'])
quarter['q1'] = (df['quarter'] == 1).astype(int)
quarter['q2'] = (df['quarter'] == 2).astype(int)
quarter['q3'] = (df['quarter'] == 3).astype(int)
quarter['q4'] = (df['quarter'] == 4).astype(int)
quarter['q5'] = (df['quarter'] == 5).astype(int)
quarter

,q1,q2,q3,q4,q5
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
1192,1,0,0,0,0
1193,1,0,0,0,0
1194,1,0,0,0,0
1195,1,0,0,0,0


In [265]:
depart=pd.DataFrame(columns=['d0','d1'])
depart['d0']=(df['department']==0).astype(int)
depart['d1']=(df['department']==1).astype(int)
depart

,d0,d1
0,1,0
1,0,1
2,1,0
3,1,0
4,1,0
...,...,...
1192,0,1
1193,0,1
1194,0,1
1195,0,1


In [266]:



day=pd.DataFrame(columns=['t0','t1','t2','t3','t4','t5'])
day['t0']=(df['day']==0).astype(int)
day['t1']=(df['day']==1).astype(int)
day['t2']=(df['day']==2).astype(int)
day['t3']=(df['day']==3).astype(int)
day['t4']=(df['day']==4).astype(int)
day['t5']=(df['day']==5).astype(int)
day

,t0,t1,t2,t3,t4,t5
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0
...,...,...,...,...,...,...
1192,0,0,0,0,0,1
1193,0,0,0,0,0,1
1194,0,0,0,0,0,1
1195,0,0,0,0,0,1


In [267]:

team = pd.DataFrame(columns=['dy0','dy1','dy2','dy3','dy4','dy5','dy6','dy7','dy8','dy9','dy10','dy11'])

team['dy0']  = (df['team'] == 0).astype(int)
team['dy1']  = (df['team'] == 1).astype(int)
team['dy2']  = (df['team'] == 2).astype(int)
team['dy3']  = (df['team'] == 3).astype(int)
team['dy4']  = (df['team'] == 4).astype(int)
team['dy5']  = (df['team'] == 5).astype(int)
team['dy6']  = (df['team'] == 6).astype(int)
team['dy7']  = (df['team'] == 7).astype(int)
team['dy8']  = (df['team'] == 8).astype(int)
team['dy9']  = (df['team'] == 9).astype(int)
team['dy10'] = (df['team'] == 10).astype(int)
team['dy11'] = (df['team'] == 11).astype(int)
team

,dy0,dy1,dy2,dy3,dy4,dy5,dy6,dy7,dy8,dy9,dy10,dy11
0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1192,0,0,0,0,0,0,0,0,0,1,0,0
1193,1,0,0,0,0,0,0,0,0,0,0,0
1194,0,0,0,0,0,1,0,0,0,0,0,0
1195,0,0,0,0,0,0,0,0,1,0,0,0


In [268]:
merged = pd.concat([quarter, depart, day, team], axis=1)
merged['satisfied']=df['satisfied']
train_df = merged.iloc[:int(len(merged)*0.8)]       # first 80%
test_df  = merged.iloc[int(len(merged)*0.8):]
print(train_df.shape)
print(test_df.shape)

(957, 26)
(240, 26)


In [269]:
train_df

,q1,q2,q3,q4,q5,d0,d1,t0,t1,t2,...,dy3,dy4,dy5,dy6,dy7,dy8,dy9,dy10,dy11,satisfied
0,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,True
1,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,True
2,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,True
3,0,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,True
4,0,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,0,0,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,True
953,0,0,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,True
954,0,0,1,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,False
955,0,0,1,0,0,0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,False


In [270]:
train_df.iloc[:,:-1]

,q1,q2,q3,q4,q5,d0,d1,t0,t1,t2,...,dy2,dy3,dy4,dy5,dy6,dy7,dy8,dy9,dy10,dy11
0,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,0,0,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
953,0,0,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
954,0,0,1,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
955,0,0,1,0,0,0,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [271]:
(train_df.iloc[:,-1]).unique()

array([ True, False])

### Exercise 3.2 - SVM with Different Kernels (15 points)

Using all the attributes we have, please build a SVM that predicts the column `satisfied`. <br >
Specifically, please
 - Build one SVM with **linear kernel**.
 - Build another SVM but with **rbf kernel**.
 - Report the **testing results** of **both models** using `classification report`.

The kernel is the only setting requirement. <br >
Other hyperparameter tuning is not required. But make sure they are the same in these two SVMs if you'd like to tune the model. In other words, the only difference between the two SVMs should be the kernel setting.

**Remember to scale your data. The scaling method is up to you.**

In [272]:
# Remember to do this task with your processed data from Exercise 3.1
X=np.array(train_df.iloc[:,:-1])
y=np.array(train_df.iloc[:,-1])
w=np.zeros(X.shape[1])
b=0.0
alpha = 0.01 
lam = 0.001 
iterations = 20
for i in range(iterations):  
    margin= y * (X @ w + b)
    new_w=lam * w - (1.0/(X.shape[0])) * (X[margin<1].T @ y[margin<1])
    new_b= -(1.0/(X.shape[0])) * np.sum(y[margin<1])
    w = w - alpha * new_w
    b = b - alpha * new_b


In [273]:
yhat_train= np.where((X @ w + b) >= 0, 1, -1)
yhat_train

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [274]:
from sklearn.metrics import classification_report
print(classification_report(y, yhat_train))


              precision    recall  f1-score   support

       False       0.00      0.00      0.00       257
        True       0.73      1.00      0.84       700

    accuracy                           0.73       957
   macro avg       0.37      0.50      0.42       957
weighted avg       0.54      0.73      0.62       957



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [275]:
# Remember to do this task with your processed data from Exercise 3.1
X=np.array(test_df.iloc[:,:-1])
y=np.array(test_df.iloc[:,-1])
w=np.zeros(X.shape[1])
b=0.0
alpha = 0.01 
lam = 0.001 
iterations = 20
for i in range(iterations):  
    margin= y * (X @ w + b)
    new_w=lam * w - (1.0/(X.shape[0])) * (X[margin<1].T @ y[margin<1])
    new_b= (1.0/(X.shape[0])) * np.sum(y[margin<1])
    w = w - alpha * new_w
    b = b - alpha * new_b


In [276]:
yhat_test= np.where((X @ w + b) >= 0, 1, -1)
yhat_test

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1, -1, -1, -1,
       -1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1,
       -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1

In [277]:
print(classification_report(y, yhat_test))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       0.00      0.00      0.00        65
           1       0.85      0.44      0.58       175

    accuracy                           0.32       240
   macro avg       0.28      0.15      0.19       240
weighted avg       0.62      0.32      0.42       240



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_pr

In [278]:
X=np.array(train_df.iloc[:,:-1])
y=np.array(train_df.iloc[:,-1])
a=np.zeros(X.shape[0])
b=0.0
C = 1.0
gamma = 0.5
tolerence = 1e-3
ker= np.diag(X @ X.T)[:,None]
K=np.exp( -gamma *(ker+ker.T - 2* (X@X.T)))
yhat_train=K @ (a * y) + b 


In [279]:
print(classification_report(y, yhat_train))

              precision    recall  f1-score   support

       False       0.27      1.00      0.42       257
        True       0.00      0.00      0.00       700

    accuracy                           0.27       957
   macro avg       0.13      0.50      0.21       957
weighted avg       0.07      0.27      0.11       957



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [280]:
X=np.array(test_df.iloc[:,:-1])
y=np.array(test_df.iloc[:,-1])
a=np.zeros(X.shape[0])
b=0.0
gamma = 0.5
ker= np.diag(X @ X.T)[:,None]
K=np.exp( -gamma *(ker+ker.T - 2* (X@X.T)))
yhat_test=K @ (a * y) + b 


In [281]:
print(classification_report(y, yhat_test))

              precision    recall  f1-score   support

       False       0.27      1.00      0.43        65
        True       0.00      0.00      0.00       175

    accuracy                           0.27       240
   macro avg       0.14      0.50      0.21       240
weighted avg       0.07      0.27      0.12       240



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [282]:
train_df

,q1,q2,q3,q4,q5,d0,d1,t0,t1,t2,...,dy3,dy4,dy5,dy6,dy7,dy8,dy9,dy10,dy11,satisfied
0,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,True
1,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,True
2,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,True
3,0,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,True
4,0,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,0,0,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,True
953,0,0,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,True
954,0,0,1,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,False
955,0,0,1,0,0,0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,False


In [283]:
train_df.columns[:-1]

Index(['q1', 'q2', 'q3', 'q4', 'q5', 'd0', 'd1', 't0', 't1', 't2', 't3', 't4',
       't5', 'dy0', 'dy1', 'dy2', 'dy3', 'dy4', 'dy5', 'dy6', 'dy7', 'dy8',
       'dy9', 'dy10', 'dy11'],
      dtype='object')

### Exercise 3.3 - SVM with Over-sampling (10 points)
 - For the column `satisfied` in our **training set**, please **print out** the frequency of each class.
 - Oversample the **training data**.
 - For the column `satisfied` in the oversampled data, **print out** the frequency of each class  again.
 - Re-build the 2 SVMs with the same setting you have in Exercise 3.2, but **use oversampled training data** instead.
     - Do not forget to scale the data first. As always, the scaling method is up to you.
 - Report the **testing result** with `classification_report`.

You can use ANY methods listed on [here](https://imbalanced-learn.org/stable/references/over_sampling.html#) such as RandomOverSampler or SMOTE. <br >
You are definitely welcomed to build your own oversampler. <br >

Note that you do not have to over-sample your testing data.

In [284]:
# Remember to do this task with your processed data from Exercise 3.1

## 4) Collaborative Statement (5 points)
#### You must fill this out even if you worked alone to get credit.

It is mandatory to include a Statement of Collaboration in each submission, that follows the guidelines below.
Include the names of everyone involved in the discussions (especially in-person ones), and what was discussed.
All students are required to follow the academic honesty guidelines posted on the course website. For
programming assignments in particular, I encourage students to organize (perhaps using Piazza) to discuss the
task descriptions, requirements, possible bugs in the support code, and the relevant technical content before they
start working on it. However, you should not discuss the specific solutions, and as a guiding principle, you are
not allowed to take anything written or drawn away from these discussions (no photographs of the blackboard,
written notes, referring to Piazza, etc.). Especially after you have started working on the assignment, try to restrict
the discussion to Piazza as much as possible, so that there is no doubt as to the extent of your collaboration.

Even if you did not use any outside resources or collaborate with anyone, please state that explicitly in the space below.